In [ ]:
import pandas as pd
import numpy as np
from numpy import arange
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import mean_absolute_error # 평가기준 
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso, LassoLarsIC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

In [ ]:
data1 = pd.read_csv("data_final.csv")
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 60 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     164 non-null    int64  
 1   name                     164 non-null    object 
 2   tot_f_safety             123 non-null    float64
 3   dept_safety              123 non-null    float64
 4   crime_safety             123 non-null    float64
 5    rob_satety_q1           123 non-null    float64
 6   mur_safety_q2            123 non-null    float64
 7   ta_safety_q3             123 non-null    float64
 8   raw_odder_q4             123 non-null    float64
 9   overall_q5               123 non-null    float64
 10  polis_num                164 non-null    float64
 11  popu_num                 164 non-null    float64
 12  local_num                164 non-null    float64
 13  popu_male_num            164 non-null    float64
 14  popu_female_num          1

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
result = le.fit_transform(data1['name'])
print(result)
data1['polis'] = result

[31 28 10 17 32 26 21 13 15 25 20 14  7 18  4  9 30  2  6  5  3 29  8 19
 24 22 11 16 27 12 23 36 34 35 33 40 39  1  0 37 38 31 28 10 17 32 26 21
 13 15 25 20 14  7 18  4  9 30  2  6  5  3 29  8 19 24 22 11 16 27 12 23
 36 34 35 33 40 39  1  0 37 38 31 28 10 17 32 26 21 13 15 25 20 14  7 18
  4  9 30  2  6  5  3 29  8 19 24 22 11 16 27 12 23 36 34 35 33 40 39  1
  0 37 38  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20
 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]


In [ ]:
# 새로운 변수 생성

data1['cctv_wt_traffic'] = data1.cctv_sum - data1.cctv_traffic

data1['occur_sexual'] = data1.occur_sexualassault + data1.occur_prostitution
data1['arrest_sexual'] = data1.arrest_prostitution + data1.arrest_sexualassault

data1['total_acc'] = data1.acc_car_car + data1.acc_car + data1.acc_car_person

data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 65 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     164 non-null    int64  
 1   name                     164 non-null    object 
 2   tot_f_safety             123 non-null    float64
 3   dept_safety              123 non-null    float64
 4   crime_safety             123 non-null    float64
 5    rob_satety_q1           123 non-null    float64
 6   mur_safety_q2            123 non-null    float64
 7   ta_safety_q3             123 non-null    float64
 8   raw_odder_q4             123 non-null    float64
 9   overall_q5               123 non-null    float64
 10  polis_num                164 non-null    float64
 11  popu_num                 164 non-null    float64
 12  local_num                164 non-null    float64
 13  popu_male_num            164 non-null    float64
 14  popu_female_num          1

In [ ]:
# 
scaler = StandardScaler()
non_col = ['polis', 'lowedu']
non_scale = data1[non_col]
data1 = data1.drop(non_col, axis=1)
data1.iloc[:,10:] = scaler.fit_transform(data1.iloc[:,10:])
data1 = pd.concat([data1, non_scale], axis=1)
data1.head()

,year,name,tot_f_safety,dept_safety,crime_safety,rob_satety_q1,mur_safety_q2,ta_safety_q3,raw_odder_q4,overall_q5,...,arrest_sexualassault,arrest_prostitution,arrest_total,mean_area,cctv_wt_traffic,occur_sexual,arrest_sexual,total_acc,polis,lowedu
0,2017,서울중부경찰서,72.50,71.90,77.40,76.85,77.85,70.60,67.15,73.90,...,-0.938796,-0.161485,-1.093910,-0.507991,-0.700172,-0.073373,-0.851104,NaN,31,0.079870
1,2017,서울종로경찰서,71.60,71.05,79.95,79.05,80.85,69.50,62.80,73.00,...,-0.795974,0.620318,-1.278963,-0.472992,-1.021155,-0.107852,-0.441740,NaN,28,0.075606
2,2017,서울남대문경찰서,69.25,68.90,76.95,76.35,77.50,68.95,60.15,70.15,...,-0.938796,-0.356936,-1.329888,-0.546758,-1.073206,-0.452641,-0.923344,NaN,10,0.079870
3,2017,서울서대문경찰서,72.45,71.95,78.95,78.05,79.80,68.75,67.55,73.60,...,-0.496047,1.206671,-0.848607,-0.418945,-0.348825,-0.064753,0.027824,NaN,17,0.074332
4,2017,서울혜화경찰서,72.55,71.65,79.30,78.30,80.30,69.90,65.10,74.75,...,-0.981643,-0.487237,-1.085303,-0.531701,-1.012480,-0.219908,-1.007625,NaN,32,0.075606


In [ ]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 65 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     164 non-null    int64  
 1   name                     164 non-null    object 
 2   tot_f_safety             123 non-null    float64
 3   dept_safety              123 non-null    float64
 4   crime_safety             123 non-null    float64
 5    rob_satety_q1           123 non-null    float64
 6   mur_safety_q2            123 non-null    float64
 7   ta_safety_q3             123 non-null    float64
 8   raw_odder_q4             123 non-null    float64
 9   overall_q5               123 non-null    float64
 10  polis_num                164 non-null    float64
 11  popu_num                 164 non-null    float64
 12  local_num                164 non-null    float64
 13  popu_male_num            164 non-null    float64
 14  popu_female_num          1

In [ ]:
# stepwise_selection 함수 정의
import statsmodels.api as sm

def stepwise_selection(data, target, cutoff=0.01) :
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0) :
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features :
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value < cutoff) : # 선택과정
            best_features.append(new_pval.idxmin())
            while (len(best_features)>0) :
                best_features_with_constant = sm.add_constant(data[best_features])
                p_values = sm.OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value = p_values.max()
                if(max_p_value >= cutoff) : # 제거과정
                    excluded_feature = p_values.idxmax()
                    best_features.remove(excluded_feature)
                else :
                    break
        else :
            break
    return best_features

In [ ]:
# 인구변수에 대해 stepwise selection을 진행, 모델 별로 가장 설명력이 높을 것으로 예상되는 인구변수를 추림
# 절도폭력
rob = data1[['polis_num', 'popu_num', 'lowedu',
        'popu_female_num','vulner_popu_num',
       'single_person_hh_female']]

# 강도살인
mur = data1[['polis_num','popu_num', 'lowedu',
       'popu_female_num', 'vulner_popu_num',
       'single_person_hh_female']]

# 교통
ta = data1[['polis_num','popu_num', 'lowedu',
       'popu_female_num', 'vulner_popu_num',
       'single_person_hh_female']]

# 법
law = data1[['polis_num','popu_num', 'lowedu',
       'popu_female_num', 'vulner_popu_num', 
       'single_person_hh_female']]

# 전반 
# 전반안전도에 있어서는 범죄 변수도 다수가 포함되는데 이는 다중공선성 때문에 예측력을 저하시킬 것으로 예상해 전반 안전도
# 모델에 대해서만 범죄 변수에 대해서도 stepwise selection을 수행함
overall = data1[['polis_num','popu_num', 'lowedu',
       'popu_female_num', 'vulner_popu_num',
       'single_person_hh_female']]

overall2 = data1[['occur_theft', 'occur_violence',
        'occur_burglar', 'occur_murder', 'occur_traffic', 'occur_misdemeanor',
        'occur_assembly', 'occur_ad', 'occur_life', 'arrest_theft', 'arrest_violence',
        'arrest_burglar', 'arrest_murder', 'arrest_traffic',
        'arrest_misdemeanor', 'arrest_assembly', 'arrest_ad', 'arrest_life',
        'occur_sexual', 'arrest_sexual']]

In [ ]:
# 변수 선택 - 절도폭력
q1 = data1[' rob_satety_q1'].iloc[0:123]
q1 = q1.reset_index(drop=True)


rob = rob.iloc[0:123]
rob = rob.reset_index(drop=True)

rob_var = stepwise_selection(rob, q1, cutoff=0.1)
print(rob_var) 

['lowedu', 'popu_num', 'vulner_popu_num', 'popu_female_num']


/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to sil

In [ ]:
# 변수 선택 - 강도살인
q2 = data1['mur_safety_q2'].iloc[0:123]
q2 = q2.reset_index(drop=True)


mur = mur.iloc[0:123]
mur = mur.reset_index(drop=True)

mur_var = stepwise_selection(mur, q2, cutoff=0.05)
print(mur_var)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


['lowedu', 'popu_num', 'vulner_popu_num', 'popu_female_num']


/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


In [ ]:
# 변수 선택 - 교통
q3 = data1['ta_safety_q3'].iloc[41:123] # 2017년은 제외하고 이용
q3 = q3.reset_index(drop=True)


traffic = ta.iloc[41:123]
traffic = traffic.reset_index(drop=True)

traf_var = stepwise_selection(traffic, q3, cutoff=0.05)
print(traf_var)

[]


/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


In [ ]:
# 변수 선택 - 법
q4 = data1['raw_odder_q4'].iloc[0:123]
q4 = q4.reset_index(drop=True)

law = law.iloc[0:123]
law = law.reset_index(drop=True)


law_var = stepwise_selection(law, q4, cutoff=0.05)
print(law_var)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


['lowedu', 'polis_num']


/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


In [ ]:
# 변수 선택 - 전반
q5 = data1['overall_q5'].iloc[41:123]
q5 = q5.reset_index(drop=True)

overall1 = overall.iloc[41:123]
overall1 = overall1.reset_index(drop=True)

overall22 = overall2.iloc[41:123]
overall22 = overall22.reset_index(drop=True)

overall_var1 = stepwise_selection(overall1, q5, cutoff=0.05)
print(overall_var1)


# 전반안전도에 들어가는 범죄 변수의 경우 그 종류가 너무 많고 다중공선성이 심해 stepwise를 범죄변수에도 적용
overall_var2 = stepwise_selection(overall22,q5, cutoff=0.05) 
print(overall_var2)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


['lowedu', 'single_person_hh_female']


/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to sil

['occur_misdemeanor', 'arrest_traffic', 'arrest_assembly', 'occur_traffic', 'arrest_sexual', 'occur_murder', 'arrest_theft']


/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


In [ ]:
# 절도폭력
rob = data1[['year','polis', 'mean_area', 'cctv_wt_traffic', 'bell','light', 'drink', 'occur_theft', 'occur_violence',
             'occur_sexual', 'arrest_sexual', 'arrest_theft', 'arrest_violence'] + rob_var + [' rob_satety_q1']] 

# 강도살인
mur = data1[['year','polis','mean_area','cctv_wt_traffic', 'bell','light', 'drink', 
             'occur_burglar', 'occur_murder', 'arrest_burglar', 'arrest_murder', 'occur_sexual', 'arrest_sexual']+mur_var+
            ['mur_safety_q2']]

# 교통
ta = data1[['year','polis','mean_area','cctv_traffic', 'drink', 'occur_traffic','arrest_traffic','total_acc',] + traf_var+
           ['ta_safety_q3']]

# 법
law = data1[['year','polis','cctv_sum', 'light', 'drink','bell','suicide','mean_area', 'occur_misdemeanor','occur_assembly', 'occur_ad', 
             'occur_life', 'arrest_misdemeanor', 'arrest_assembly', 
             'arrest_ad', 'arrest_life'] + law_var + ['raw_odder_q4']]

# 전반
overall = data1[['year','polis','cctv_sum', 'total_acc','mean_area', 'bell','light', 'drink',  'fire', 'suicide']
                +overall_var1 + overall_var2 + ['overall_q5']]

In [ ]:
# 라쏘
lasso = Lasso()
grid = dict()
grid['alpha'] = arange(0,1, 0.005)

search = GridSearchCV(lasso, grid, scoring = 'neg_mean_absolute_error', n_jobs=-1)

In [ ]:
# 앙상블에 사용할 라쏘 모델 20개
c=20

## 절도폭력

In [ ]:
# rob
# train - test 나누기
train_x = rob.iloc[0:-41,0:-1]
#train_x = train_x.drop('name', axis=1)

train_y = rob.iloc[0:-41,-1]

test_x = rob.iloc[-41:,0:-1]
#test_x = test_x.drop('name', axis=1)
#police = rob.loc[rob.year==2020, 'name']

ab = search.fit(train_x, train_y)

# 앙상블(배깅)
cv_idx = list(np.random.choice(c, len(train_x) , replace=True))
train_x['cv_idx'] = cv_idx
train_y_=pd.DataFrame(train_y)
train_y_['cv_idx'] = cv_idx

las = Lasso(alpha=list(ab.best_params_.values())[0]) # 찾은 bests parameter값을 대입
a = pd.DataFrame(columns=list(train_x.columns))
a = a.drop('cv_idx', axis=1)
b = list()
for idx in range(c) :
    train_x_idx = train_x[train_x.cv_idx != idx]
    train_x_idx = train_x_idx.drop('cv_idx', axis=1)
    train_y_idx = train_y_[train_y_.cv_idx != idx]
    Model = las.fit(train_x_idx, train_y_idx.iloc[:,0])
    a.loc[idx]=list(Model.coef_)
    b.append(Model.intercept_)
    
y_pred1 = pd.DataFrame({'rob':(test_x * a.sum()/c).sum(axis=1) + sum(b)/c})
y_pred1

/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/opt/app-root/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 221.08367469013308, tolerance: 0.10515033673469389
  positive)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/l

,rob
123,78.595439
124,77.351526
125,80.967315
126,80.657250
127,76.120676
128,78.765701
129,74.769563
130,77.571469
131,76.489099
132,74.800615


In [ ]:
a.sum()/c

year                 0.587570
polis               -0.027670
mean_area            1.081255
cctv_wt_traffic     -0.169079
bell                -0.326769
light                0.120966
drink               -0.149819
occur_theft          1.454118
occur_violence      -2.196002
occur_sexual         1.489927
arrest_sexual       -0.775519
arrest_theft        -0.689987
arrest_violence      0.412432
lowedu            -109.514486
popu_num           -17.686210
vulner_popu_num      2.732789
popu_female_num     14.316328
dtype: float64

## 강도살인

In [ ]:
# mur
# train - test 나누기
train_x = mur.iloc[0:-41,0:-1]
#train_x = train_x.drop('name', axis=1)

train_y = mur.iloc[0:-41,-1]

test_x = mur.iloc[-41:,0:-1]
#test_x = test_x.drop('name', axis=1)
#police = mur.loc[mur.year==2020, 'name']

ab = search.fit(train_x, train_y)

# 앙상블(배깅)
cv_idx = list(np.random.choice(c, len(train_x) , replace=True))
train_x['cv_idx'] = cv_idx
train_y_=pd.DataFrame(train_y)
train_y_['cv_idx'] = cv_idx

las = Lasso(alpha=list(ab.best_params_.values())[0])
a = pd.DataFrame(columns=list(train_x.columns))
a = a.drop('cv_idx', axis=1)
b = list()
for idx in range(c) :
    train_x_idx = train_x[train_x.cv_idx != idx]
    train_x_idx = train_x_idx.drop('cv_idx', axis=1)
    train_y_idx = train_y_[train_y_.cv_idx != idx]
    Model = las.fit(train_x_idx, train_y_idx.iloc[:,0])
    a.loc[idx]=list(Model.coef_)
    b.append(Model.intercept_)
    
y_pred2 = pd.DataFrame({'murder':(test_x * a.sum()/c).sum(axis=1) + sum(b)/c})
y_pred2

/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/opt/app-root/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 255.80815564510218, tolerance: 0.09895312499999992
  positive)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/l

,murder
123,80.138148
124,78.545857
125,82.688520
126,81.579075
127,78.849654
128,80.954540
129,76.771805
130,78.870611
131,78.417603
132,76.641422


In [ ]:
a.sum()/c

year                0.634514
polis              -0.003451
mean_area           0.751334
cctv_wt_traffic    -0.121327
bell               -0.380032
light               0.392359
drink               0.073407
occur_burglar       0.010451
occur_murder        0.103205
arrest_burglar      0.217386
arrest_murder      -0.357541
occur_sexual       -0.065450
arrest_sexual      -0.396795
lowedu            -99.951488
popu_num          -19.935209
vulner_popu_num     3.039844
popu_female_num    16.507085
dtype: float64

## 교통

In [ ]:
# ta
# train - test 나누기
train_x = ta.iloc[42:-41,0:-1]
#train_x = train_x.drop('name', axis=1)

train_y = ta.iloc[42:-41,-1]

test_x = ta.iloc[-41:,0:-1]
#test_x = test_x.drop('name', axis=1)
#police = ta.loc[ta.year==2020, 'name']


ab = search.fit(train_x, train_y)

# 앙상블(배깅)
cv_idx = list(np.random.choice(c, len(train_x) , replace=True))
train_x['cv_idx'] = cv_idx
train_y_=pd.DataFrame(train_y)
train_y_['cv_idx'] = cv_idx

las = Lasso(alpha=list(ab.best_params_.values())[0])
a = pd.DataFrame(columns=list(train_x.columns))
a = a.drop('cv_idx', axis=1)
b = list()
for idx in range(c) :
    train_x_idx = train_x[train_x.cv_idx != idx]
    train_x_idx = train_x_idx.drop('cv_idx', axis=1)
    train_y_idx = train_y_[train_y_.cv_idx != idx]
    Model = las.fit(train_x_idx, train_y_idx.iloc[:,0])
    a.loc[idx]=list(Model.coef_)
    b.append(Model.intercept_)
    
y_pred3 = pd.DataFrame({'traffic':(test_x * a.sum()/c).sum(axis=1) + sum(b)/c})
y_pred3

/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/opt/app-root/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 105.89980990904516, tolerance: 0.03412025000000003
  positive)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/l

,traffic
123,71.096488
124,70.056593
125,70.932591
126,71.023734
127,71.533665
128,71.121247
129,70.589184
130,72.414922
131,71.727273
132,71.385083


In [ ]:
a.sum()/c

year              1.734849
polis             0.003042
mean_area        -0.299021
cctv_traffic      0.573301
drink            -0.439962
occur_traffic    -0.357272
arrest_traffic    0.000000
total_acc         0.016462
dtype: float64

## 법질서

In [ ]:
# law
# train - test 나누기
train_x = law.iloc[0:-41,0:-1]
#train_x = train_x.drop('name', axis=1)

train_y = law.iloc[0:-41,-1]

test_x = law.iloc[-41:,0:-1]
#test_x = test_x.drop('name', axis=1)
#police = law.loc[law.year==2020, 'name']

ab = search.fit(train_x, train_y)

# 앙상블(배깅)
cv_idx = list(np.random.choice(c, len(train_x) , replace=True))
train_x['cv_idx'] = cv_idx
train_y_=pd.DataFrame(train_y)
train_y_['cv_idx'] = cv_idx

las = Lasso(alpha=list(ab.best_params_.values())[0])
a = pd.DataFrame(columns=list(train_x.columns))
a = a.drop('cv_idx', axis=1)
b = list()
for idx in range(c) :
    train_x_idx = train_x[train_x.cv_idx != idx]
    train_x_idx = train_x_idx.drop('cv_idx', axis=1)
    train_y_idx = train_y_[train_y_.cv_idx != idx]
    Model = las.fit(train_x_idx, train_y_idx.iloc[:,0])
    a.loc[idx]=list(Model.coef_)
    b.append(Model.intercept_)
    
y_pred4 = pd.DataFrame({'law':(test_x * a.sum()/c).sum(axis=1) + sum(b)/c})
y_pred4

/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/opt/app-root/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 225.23963125243688, tolerance: 0.10629567346938776
  positive)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/l

,law
123,72.838445
124,71.288291
125,77.249758
126,73.506912
127,70.066169
128,75.682249
129,72.588962
130,77.643105
131,72.014713
132,71.587482


In [ ]:
a.sum()/c

year                   2.808607
polis                 -0.034425
cctv_sum               0.171732
light                 -0.435200
drink                 -0.579696
suicide               -0.436711
mean_area              1.369918
occur_misdemeanor      1.030740
occur_assembly         1.127275
occur_ad               0.429971
occur_life             0.557100
arrest_misdemeanor    -1.821086
arrest_assembly       -1.018274
arrest_ad             -0.571478
arrest_life            0.065271
lowedu               -80.787519
polis_num              0.477698
dtype: float64

## 전반

In [ ]:
# overall
# train - test 나누기
train_x = overall.iloc[42:-41,0:-1]
#train_x = train_x.drop('name', axis=1)

train_y = overall.iloc[42:-41,-1]

test_x = overall.iloc[-41:,0:-1]
#test_x = test_x.drop('name', axis=1)
#police = overall.loc[overall.year==2020, 'name']

ab = search.fit(train_x, train_y)

# 앙상블(배깅)
cv_idx = list(np.random.choice(c, len(train_x) , replace=True))
train_x['cv_idx'] = cv_idx
train_y_=pd.DataFrame(train_y)
train_y_['cv_idx'] = cv_idx

las = Lasso(alpha=list(ab.best_params_.values())[0])
a = pd.DataFrame(columns=list(train_x.columns))
a = a.drop('cv_idx', axis=1)
b = list()
for idx in range(c) :
    train_x_idx = train_x[train_x.cv_idx != idx]
    train_x_idx = train_x_idx.drop('cv_idx', axis=1)
    train_y_idx = train_y_[train_y_.cv_idx != idx]
    Model = las.fit(train_x_idx, train_y_idx.iloc[:,0])
    a.loc[idx]=list(Model.coef_)
    b.append(Model.intercept_)
    
y_pred5 = pd.DataFrame({'overall':(test_x * a.sum()/c).sum(axis=1) + sum(b)/c})
y_pred5

/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/opt/app-root/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 68.08393394846428, tolerance: 0.049358839843749995
  positive)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/l

,overall
123,75.796268
124,74.486582
125,80.130341
126,77.365255
127,76.016344
128,71.509209
129,70.500644
130,77.257664
131,74.828731
132,73.326120


In [ ]:
a.sum()/c

year                        1.995446
polis                      -0.018888
cctv_sum                    0.584195
total_acc                   0.327979
mean_area                   0.049968
bell                       -0.521363
light                      -0.076541
drink                      -0.182769
fire                        1.126498
suicide                    -0.436702
lowedu                    -36.260874
single_person_hh_female    -0.197474
occur_misdemeanor          -1.401370
arrest_traffic              0.418481
arrest_assembly             0.690424
occur_traffic               1.369073
arrest_sexual              -1.227619
occur_murder               -0.202289
arrest_theft               -0.834644
dtype: float64

## 제출 포맷으로 변경

In [ ]:
police = list(data1.iloc[-41:].name.values)
police = pd.DataFrame(police)
police = police.reset_index()
police = police.drop('index', axis=1)
police

,0
0,마산동부경찰서
1,마산중부경찰서
2,서울강남경찰서
3,서울강동경찰서
4,서울강북경찰서
5,서울강서경찰서
6,서울관악경찰서
7,서울광진경찰서
8,서울구로경찰서
9,서울금천경찰서


In [ ]:
pred = pd.concat([y_pred1, y_pred2, y_pred3, y_pred4, y_pred5], axis=1)
pred = pred.reset_index(drop=True)

In [ ]:
pred = pd.concat([police, pred], axis=1)
pred

,0,rob,murder,traffic,law,overall
0,마산동부경찰서,78.595439,80.138148,71.096488,72.838445,75.796268
1,마산중부경찰서,77.351526,78.545857,70.056593,71.288291,74.486582
2,서울강남경찰서,80.967315,82.688520,70.932591,77.249758,80.130341
3,서울강동경찰서,80.657250,81.579075,71.023734,73.506912,77.365255
4,서울강북경찰서,76.120676,78.849654,71.533665,70.066169,76.016344
5,서울강서경찰서,78.765701,80.954540,71.121247,75.682249,71.509209
6,서울관악경찰서,74.769563,76.771805,70.589184,72.588962,70.500644
7,서울광진경찰서,77.571469,78.870611,72.414922,77.643105,77.257664
8,서울구로경찰서,76.489099,78.417603,71.727273,72.014713,74.828731
9,서울금천경찰서,74.800615,76.641422,71.385083,71.587482,73.326120


In [ ]:
pred.set_index(0, inplace=True)
pred

,rob,murder,traffic,law,overall
0,,,,,
마산동부경찰서,78.595439,80.138148,71.096488,72.838445,75.796268
마산중부경찰서,77.351526,78.545857,70.056593,71.288291,74.486582
서울강남경찰서,80.967315,82.688520,70.932591,77.249758,80.130341
서울강동경찰서,80.657250,81.579075,71.023734,73.506912,77.365255
서울강북경찰서,76.120676,78.849654,71.533665,70.066169,76.016344
서울강서경찰서,78.765701,80.954540,71.121247,75.682249,71.509209
서울관악경찰서,74.769563,76.771805,70.589184,72.588962,70.500644
서울광진경찰서,77.571469,78.870611,72.414922,77.643105,77.257664
서울구로경찰서,76.489099,78.417603,71.727273,72.014713,74.828731


In [ ]:
jechul = pd.read_csv("input/25.결과제출양식.csv")

je = pd.DataFrame(jechul['경찰서명'])
je.columns = ['name']
je.name = je.name.apply(lambda x : x.lstrip())
je.name = je.name.apply(lambda x : x.rstrip())
je = je.name.to_list()

jejeje = pred.reindex(index=je)
jejeje['bum'] = (jejeje.rob+jejeje.murder)/2
jejeje['bunya'] = (jejeje.bum * 0.343) + (jejeje.traffic * 0.305) + (jejeje.law * 0.352)
jejeje['jonghab'] = (jejeje.overall * 0.3) + (jejeje.bunya*0.7)

col = ['jonghab','bunya','bum','rob','murder', 'traffic', 'law', 'overall']
jejeje = jejeje[col]

jechul.iloc[:,2:] = jejeje.iloc[:,0:].values
jechul

,순번,경찰서명,종합체감안전도_점수 \n(전반적안전도*0.3) + (분야별안전도*0.7),분야별 안전도\n(범죄안전도*0.343) + \n(교통사고안전도*0.305)\n + (법질서준수도*0.352),범죄 안전도\n(절도폭력안전도+강도살인안전도)/2,문항1. 절도 폭력안전도\n (문항1 점수 * 10),문항2. 강도 살인 안전도\n(문항2 점수 * 10),문항3. 교통사고 안전도\n(문항3 점수*10),문항4. 법질서 준수도\n(문항 4점수 * 10),문항5. 전반적 안전도\n(문항 5점수 * 10)
0,1,서울중부경찰서,75.031822,74.583329,78.944254,78.052319,79.836190,71.993108,72.578273,76.078305
1,2,서울종로경찰서,77.664711,76.792736,79.194611,78.131866,80.257356,71.974773,78.626928,79.699321
2,3,서울남대문경찰서,75.635315,75.080316,78.946806,78.553484,79.340127,72.041295,73.945929,76.930311
3,4,서울서대문경찰서,76.092885,75.450793,80.436910,78.808766,82.065054,72.100436,73.495169,77.591102
4,5,서울혜화경찰서,75.620357,74.973313,79.510239,78.578962,80.441517,72.115835,73.028329,77.130125
5,6,서울용산경찰서,74.493492,74.332327,79.129388,77.826463,80.432313,71.601942,72.023735,74.869544
6,7,서울성북경찰서,75.439521,74.851044,78.562554,76.981115,80.143994,72.178098,73.550478,76.812633
7,8,서울동대문경찰서,74.011754,73.432716,78.319432,76.691097,79.947768,71.509755,70.337145,75.362844
8,9,서울마포경찰서,76.333508,75.391845,80.529402,79.969374,81.089429,71.624591,73.649885,78.530723
9,10,서울영등포경찰서,75.597774,74.498212,77.147273,76.670729,77.623818,72.944941,73.262755,78.163418


In [ ]:
# 최종 제출셋 내보내기
jechul.to_csv("output.csv", index=False, encoding='utf-8')